<a href="https://colab.research.google.com/github/cnovak232/DL_Speech_Enhancement/blob/main/DL_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load Data by cloning the repo - easiest way to access shared data

In [ ]:
!git clone https://github.com/cnovak232/DL_Speech_Enhancement.git


Define some helper function for plotting and playing audio

In [ ]:
import torch
import torchaudio as ta
import librosa as lib
from IPython.display import Audio, display
import matplotlib
import matplotlib.pyplot as plt

# helper functions for audio and what not
# mostly taken for torchaudio tutorials 

def play_audio(waveform, sample_rate):
    waveform = waveform.numpy()

    num_channels, num_frames = waveform.shape
    if num_channels == 1:
        display(Audio(waveform[0], rate=sample_rate))
    elif num_channels == 2:
        display(Audio((waveform[0], waveform[1]), rate=sample_rate))
    else:
        raise ValueError("Waveform with more than 2 channels are not supported.")

def plot_waveform(waveform, sample_rate, title="Waveform", xlim=None, ylim=None):
    waveform = waveform.numpy()

    num_channels, num_frames = waveform.shape
    time_axis = torch.arange(0, num_frames) / sample_rate

    figure, axes = plt.subplots(num_channels, 1)
    if num_channels == 1:
        axes = [axes]
    for c in range(num_channels):
        axes[c].plot(time_axis, waveform[c], linewidth=1)
        axes[c].grid(True)
    if num_channels > 1:
        axes[c].set_ylabel(f'Channel {c+1}')
    if xlim:
        axes[c].set_xlim(xlim)
    if ylim:
        axes[c].set_ylim(ylim)

    figure.suptitle(title)
    plt.show(block=False)

def get_spectrogram(
    waveform = None,
    n_fft = 512,
    win_len = None,
    hop_len = None,
    power = 1.0 ):
    spectrogram = ta.transforms.Spectrogram(
      n_fft=n_fft,
      win_length=win_len,
      hop_length=hop_len,
      center=True,
      pad_mode="reflect",
      power=power )
    
    return spectrogram(waveform)

def plot_spectrogram(spec, type = "amplitude", title=None, ylabel='freq_bin', aspect='auto', xmax=None):
    fig, axs = plt.subplots(1, 1)
    axs.set_title(title or 'Spectrogram (db)')
    axs.set_ylabel(ylabel)
    axs.set_xlabel('frame')
    toDb = ta.transforms.AmplitudeToDB(type)
    im = axs.imshow(spec, origin='lower', aspect=aspect)
    if xmax:
        axs.set_xlim((0, xmax))
    fig.colorbar(im, ax=axs)
    plt.show(block=False)

def norm_spec( spec ):
    normed = spec / spec.max()
    return normed, spec.max()


Define a Custom Dataset class for the Data and read it in

In [ ]:
# read dataset in and downsample / transform / pad if needed
from torch.utils.data import Dataset
import torch
import os

class VoiceBankDemand(Dataset):
    def __init__(self, clean_dir, noisy_dir, list_dir, 
                 data = "train", len_samples = None, downsample = None, 
                 transform = None ):
        self.clean_dir = clean_dir
        self.noisy_dir = noisy_dir
        self.list_dir = list_dir
        self.num_samples = len_samples
        self.downsample = downsample
        self.transform = transform
        self.data = data
    
    def __len__(self):
        return len(self.list_dir)

    def __getitem__( self, idx ):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        clean_name = os.path.join( self.clean_dir, self.list_dir[idx] )
        noisy_name = os.path.join( self.noisy_dir, self.list_dir[idx] )
        clean_audio, fs = ta.load(clean_name)
        noisy_audio, fs= ta.load(noisy_name)

        if self.downsample:
            downsampler = ta.transforms.Resample(fs,self.downsample)
            clean_audio = downsampler( clean_audio )
            noisy_audio = downsampler( noisy_audio )

        if self.num_samples:
            orig_len = clean_audio.shape[1]
            if clean_audio.shape[1] > num_samples:
                clean_audio = clean_audio[:,:num_samples]
                noisy_audio = noisy_audio[:,:num_samples]
            elif clean_audio.shape[1] < num_samples:
                pad_len = int( num_samples - clean_audio.shape[1] )
                pad = torch.zeros(1,pad_len)
                clean_audio = torch.cat((clean_audio,pad), dim=1)
                noisy_audio = torch.cat((noisy_audio,pad),dim=1)
        
        if self.data == "test":
            if self.transform:
                noisy_trnsfrm = self.transform( noisy_audio )
                clean_trnsfrm = self.transform( clean_audio )
            clean_mag,_ = norm_spec( torch.abs(clean_trnsfrm) )
            clean_audio = (clean_audio, clean_mag)
            noisy_mag, norm_val = norm_spec( torch.abs(noisy_trnsfrm) )
            noisy_phase = torch.angle(noisy_trnsfrm)
            noisy_audio = (noisy_mag, noisy_phase, noisy_audio, norm_val)
        else:
            if self.transform:
                clean_audio,_ = norm_spec( self.transform( clean_audio ) )
                noisy_audio,_ = norm_spec( self.transform( noisy_audio ) )

        sample = (clean_audio, noisy_audio, orig_len)

        return sample

train_clean_path = './DL_Speech_Enhancement/clean_trainset_28spk_wav'
train_noisy_path = './DL_Speech_Enhancement/noisy_trainset_28spk_wav'
test_clean_path  = './DL_Speech_Enhancement/clean_testset_wav'
test_noisy_path  = './DL_Speech_Enhancement/noisy_testset_wav'

list_dir_train = os.listdir(train_clean_path)
list_dir_test = os.listdir(test_clean_path)

target_fs = 16000 # downsample to 16 KHz
spectrogram = ta.transforms.Spectrogram(
    n_fft=512,
    power=1.0,
    normalized = False )
complex_spec = ta.transforms.Spectrogram(
    n_fft=512,
    power=None,
    normalized = False ) # return complex spectrum

num_samples = int( 5.0 * target_fs ) 

train_set = VoiceBankDemand( clean_dir = train_clean_path,
                             noisy_dir = train_noisy_path,
                             list_dir = list_dir_train,
                             len_samples = num_samples, # clip or pad samples to be 5s
                             downsample = target_fs, # downsample to 16Khz
                             transform = spectrogram )

# returns the mag/phase of each audio file 
test_set = VoiceBankDemand( clean_dir = test_clean_path,
                            noisy_dir = test_noisy_path,
                            list_dir = list_dir_test,
                            data = "test",
                            len_samples = num_samples,
                            downsample = target_fs,
                            transform = complex_spec )


clean, noisy, orig_len = train_set[1]
print(clean.size())
print(orig_len)

clean_test, noisy_test, orig_len = test_set[0]

noisy_mag, noisy_phase, noisy_audio = noisy_test

plot_spectrogram(noisy.squeeze())
plot_spectrogram(noisy_mag.squeeze())

In [ ]:
import torch
import torch.nn as nn

# U-Net Style Autoencoder from to start from
# https://medium.com/@sriskandaryan/autoencoders-demystified-audio-signal-denoising-32a491ab023a

class UNet(nn.Module):
    def __init__(self, chnls_in=1, chnls_out=1):
        super(UNet, self).__init__()
        self.down_conv_layer_1 = DownConvBlock(chnls_in, 64, norm=False)
        self.down_conv_layer_2 = DownConvBlock(64, 128)
        self.down_conv_layer_3 = DownConvBlock(128, 256)
        self.down_conv_layer_4 = DownConvBlock(256, 256, dropout=0.5)
        self.down_conv_layer_5 = DownConvBlock(256, 256, dropout=0.5)
        self.down_conv_layer_6 = DownConvBlock(256, 256, dropout=0.5)

        self.up_conv_layer_1 = UpConvBlock(256, 256, kernel_size=(2,3), stride=2, padding=0, dropout=0.5)# 256+256 6 5 kernel_size=(2, 3), stride=2, padding=0
        self.up_conv_layer_2 = UpConvBlock(512, 256, kernel_size=(2,3), stride=2, padding=0, dropout=0.5) # 256+256 1 4
        self.up_conv_layer_3 = UpConvBlock(512, 256, kernel_size=(2,3), stride=2, padding=0, dropout=0.5) # 2 3
        self.up_conv_layer_4 = UpConvBlock(512, 128, dropout=0.5) # 3 2
        self.up_conv_layer_5 = UpConvBlock(256, 64) # 4 1
        self.up_conv_layer_6 = UpConvBlock(512, 128)
        self.up_conv_layer_7 = UpConvBlock(256, 64)
        self.upsample_layer = nn.Upsample(scale_factor=2)
        self.zero_pad = nn.ZeroPad2d((1, 0, 1, 0))
        self.conv_layer_1 = nn.Conv2d(128, chnls_out, 4, padding='same')
        self.activation = nn.Tanh()
    
    def forward(self, x):
        enc1 = self.down_conv_layer_1(x)
        enc2 = self.down_conv_layer_2(enc1) 
        enc3 = self.down_conv_layer_3(enc2)
        enc4 = self.down_conv_layer_4(enc3)
        enc5 = self.down_conv_layer_5(enc4)
        enc6 = self.down_conv_layer_6(enc5)
 
        dec1 = self.up_conv_layer_1(enc6, enc5)
        dec2 = self.up_conv_layer_2(dec1, enc4)
        dec3 = self.up_conv_layer_3(dec2, enc3)
        dec4 = self.up_conv_layer_4(dec3, enc2)
        dec5 = self.up_conv_layer_5(dec4, enc1)

        final = self.upsample_layer(dec5)
        final = self.zero_pad(final)
        final = self.conv_layer_1(final)
        return final

class UpConvBlock(nn.Module):
    def __init__(self, ip_sz, op_sz, kernel_size=4, stride= 2, padding=1 ,dropout=0.0):
        super(UpConvBlock, self).__init__()
        self.layers = nn.ModuleList([
            nn.ConvTranspose2d(ip_sz, op_sz, kernel_size=kernel_size, stride=stride, padding=padding),
            nn.InstanceNorm2d(op_sz),
            nn.ReLU(),
        ])
        if dropout:
            self.layers += [nn.Dropout(dropout)]
    def forward(self, x, enc_ip):
        x = nn.Sequential(*(self.layers))(x)
        op = torch.cat((x, enc_ip), 1)
        return op


class DownConvBlock(nn.Module):
    def __init__(self, ip_sz, op_sz, kernel_size=4, norm=True, dropout=0.0):
        super(DownConvBlock, self).__init__()
        self.layers = nn.ModuleList([nn.Conv2d(ip_sz, op_sz, kernel_size, 2, 1)])
        if norm:
            self.layers.append(nn.InstanceNorm2d(op_sz))
        self.layers += [nn.LeakyReLU(0.2)]
        if dropout:
            self.layers += [nn.Dropout(dropout)]
    def forward(self, x):
        op = nn.Sequential(*(self.layers))(x)
        return op

Main block for training the model and testing.
Initializes all training and testing parameters. 
Performs training and testing in one big loop, testing each epoch to track any overfitting.

In [ ]:
# training (work in progress)
from torch.utils.data import DataLoader

def train_model( model, dataloader, criterion, optimizer, device ):
    model.train()

    running_loss = 0.0

    # Iterate over data.
    for clean_audio,noisey_audio,orig_len in dataloader:

        # send inputs to gpu
        clean_audio = clean_audio.to(device)
        noisey_audio = noisey_audio.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        
        with torch.enable_grad():
            # Send the noisey speech sample through network
            output = model(noisey_audio)

            # compute loss between network output and clean audio
            loss = criterion(output, clean_audio)

            # backward + optimize 
            loss.backward()
            optimizer.step()

        # statistics
        running_loss += loss.item()

    epoch_loss = running_loss / len(dataloader.dataset)

    return model, epoch_loss

def test_model( model, device, criterion, dataloader, fs = 16000, inv_transform = None ):
    model.eval() 
    inv_transfrom = inv_transform.to(device)
    cleaned_waveforms = torch.empty(0)
    cleaned_mags = torch.empty(0)
    running_loss = 0
    for clean_audio, noisy_audio, orig_len in dataloader:
        noisy_mag, noisy_phase, noisy_wav, norm_val = noisy_audio
        noisy_mag = noisy_mag.to(device)
        noisy_phase = noisy_phase.to(device)
        clean_wav, clean_mag = clean_audio
        clean_mag = clean_mag.to(device)
        
        with torch.no_grad():
            # forward
            enhanced_mag = model(noisy_mag)

            loss = criterion(enhanced_mag, clean_mag)

            complex_out = torch.polar(enhanced_mag * norm_val, noisy_phase)
            #audio_enhanced = complex_out.detach().cpu()
            
            if inv_transform:
                audio_enhanced = inv_transform(audio_enhanced, clean_wav.shape[2])

            cleaned_waveforms = torch.cat( (cleaned_waveforms, audio_enhanced), 0 )
            cleaned_mags = torch.cat( (cleaned_mags, enhanced_mag.detach().cpu()), 0 )

            running_loss += loss.item()

    test_loss = running_loss / len( dataloader.dataset )

    return cleaned_waveforms, cleaned_mags, test_loss


train_loader = DataLoader( train_set, batch_size=64 )
test_loader = DataLoader( test_set, batch_size=64 )

model = UNet()

learning_rate = 0.01
num_epochs = 10

criterion = nn.MSELoss()

optimizer = torch.optim.Adam(model.parameters(),learning_rate)

device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")

model = model.to(device)

inv_spec = ta.transforms.InverseSpectrogram(n_fft=512, normalized = False )

for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)

    model, epoch_loss = train_model( model, train_loader, criterion, optimizer, device )
    print('Train Epoch Loss = ', epoch_loss )

    cleaned_waveforms, cleaned_mag, test_loss = \
        test_model( model, device, criterion, test_loader, fs = 16000, inv_transform = inv_spec)
    print('Test Epoch Loss = ', test_loss )


In [ ]:
!pip install torchmetrics[audio]
!pip install pesq

Loop through testing samples and evaluate enhanced data against clean data

In [ ]:
from torchmetrics import SignalNoiseRatio
from pesq import pesq

SNR = SignalNoiseRatio(zero_mean=True)

snr_vals_noisy = torch.empty(len(test_set))
pesq_vals_noisy = torch.empty(len(test_set))
snr_vals = torch.empty(len(test_set))
pesq_vals = torch.empty(len(test_set))

for i in range( len(test_set) ):
    clean_audio,noisy_data, orig_len = test_set[i]
    noisy_audio = noisy_data[2]
    enhanced_audio = cleaned_waveforms[i]

    if enhanced_audio.shape[1] > orig_len:
        enhanced_audio = enhanced_audio[:,:orig_len]
        noisy_audio = noisy_audio[:,:orig_len]
        clean_audio = clean_audio[:,:orig_len]

    snr_vals[i] = SNR(enhanced_audio, clean_audio)
    pesq_vals[i] = pesq( 16000, enhanced_audio.squeeze().numpy(), clean_audio.squeeze().numpy(), "wb")

    snr_vals_noisy[i] = SNR(noisy_audio, clean_audio)
    pesq_vals_noisy[i] = pesq( 16000, noisy_audio.squeeze().numpy(), clean_audio.squeeze().numpy(), "wb")



Compute average and best SNR and PESQ improvements

In [ ]:
SNR_imp = snr_vals - snr_vals_noisy
PESQ_imp = pesq_vals - pesq_vals_noisy

avg_SNR_imp = SNR_imp.sum() / len(SNR_imp)
avg_PESQ_imp = PESQ_imp.sum() / len(PESQ_imp)

peak_SNR_imp = SNR_imp.max()
peak_snr_ind = SNR_imp.argmax()
peak_PESQ_imp = PESQ_imp.max()
peak_PESQ_ind = PESQ_imp.argmax()

print("Average SNR Improvement = ", avg_SNR_imp )
print("Average PESQ Improvement = ", avg_PESQ_imp )

print("Best SNR Improvement = ", peak_SNR_imp )
print("Best PESQ Improvment = ", peak_PESQ_imp )


fs = 16000

play_audio(test_set[303][0],fs)
play_audio(cleaned_waveforms[303],fs)
play_audio(test_set[303][1][2],fs)